In [21]:
import pickle
import numpy as np
import sys
import logging
from tqdm.auto import tqdm
from contextlib import contextmanager

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed

sys.path.append("/Users/oliver/Documents/p5control-bluefors-evaluation")
sys.path.append("/home/oliver/Documents/p5control-bluefors-evaluation")

from theory.utilities.types import NDArray64
from theory.models.ha_sym import ha_sym_nonuniform_worker


@contextmanager
def suppress_logging():
    logger = logging.getLogger()
    original_level = logger.level
    logger.setLevel(logging.CRITICAL)
    try:
        yield
    finally:
        logger.setLevel(original_level)

# generate theo data

In [62]:
# data fields
Delta_0_meV: float = 0.18
V_Delta: NDArray64 = np.concatenate(
    (
        np.arange(0, 1, 0.01),
        np.arange(1, 2, 0.02),
        np.arange(2, 2.5, 0.05),
        np.arange(2.5, 5, 0.1),
    ),
    dtype=np.float64,
)
V_mV: NDArray64 = V_Delta * Delta_0_meV

tau: NDArray64 = np.concatenate(
    (
        np.arange(0.0, 0.6, 0.05),
        np.arange(0.6, 0.9, 0.01),
        np.arange(0.9, 1.0, 0.02),
    ),
    dtype=np.float64,
)

T_K: NDArray64 = np.array([0, 0.1, 0.2, 0.3], dtype=np.float64)

Delta_Delta_0_meV: NDArray64 = np.concatenate(
    (
        np.arange(0.90, 0.95, 0.025),
        np.arange(0.95, 1.05, 0.01),
        np.arange(1.05, 1.10, 0.025),
    ),
    dtype=np.float64,
)
Delta_meV: NDArray64 = Delta_Delta_0_meV * Delta_0_meV

gamma_meV: NDArray64 = np.logspace(-4, 0, 9)

In [64]:
Delta_meV, Delta_meV.shape

(array([0.162 , 0.1665, 0.171 , 0.1728, 0.1746, 0.1764, 0.1782, 0.18  ,
        0.1818, 0.1836, 0.1854, 0.1872, 0.189 , 0.189 , 0.1935, 0.198 ]),
 (16,))

In [23]:
# generate data to compare
jobs: list[
    tuple[tuple[int, int, int, int], tuple[NDArray64, float, float, float, float, bool]]
] = []
for itau, tau_i in enumerate(tau):
    for iT, T_K_i in enumerate(T_K):
        for idel, Delta_i in enumerate(Delta_meV):
            for igam, gamma_i in enumerate(gamma_meV):
                idx = (itau, iT, idel, igam)
                args = (
                    V_mV,
                    tau_i,
                    T_K_i,
                    Delta_i,
                    gamma_i,
                    True,
                )
                jobs.append((idx, args))

n_tau = len(tau)
n_T = len(T_K)
n_Delta = len(Delta_meV)
n_gamma = len(gamma_meV)
n_V = len(V_mV)

I_nA = np.empty((n_tau, n_T, n_Delta, n_gamma, n_V))

max_workers = 8  # or 6-8 depending on your Mac / thermal tolerance :)
pbar = False

if pbar:
    p_total = tqdm(total=len(jobs), desc="(τ, T, Δ, γ)")
    p_para = tqdm(total=(n_T * n_Delta * n_gamma), desc="(T, Δ, γ)", leave=True)
    p_tau = tqdm(total=n_tau, desc="(τ)", leave=True)

with ProcessPoolExecutor(max_workers=max_workers) as ex:
    futures = {ex.submit(ha_sym_nonuniform_worker, *args): idx for idx, args in jobs}

    for fut in as_completed(futures):
        try:
            itau, iT, idel, igam = futures[fut]
            i = fut.result()
            I_nA[itau, iT, idel, igam, :] = i
        except EOFError:
            raise (fut, futures[fut])

        if pbar:
            p_total.update(1)

            p_tau.n = itau + 1
            p_tau.refresh()

            n_para = iT * n_Delta * n_gamma + idel * n_gamma + igam
            p_para.n = n_para
            p_para.refresh()

            print(itau, iT, idel, igam)
I_nA.shape

loaded constants...
loaded constants...
loaded constants...
loaded constants...
loaded constants...
loaded constants...
loaded constants...
loaded constants...


(48, 4, 16, 9, 185)

In [4]:
theo = dict(
    V_mV=V_mV,
    tau=tau,
    T_K=T_K,
    Delta_meV=Delta_meV,
    gamma_meV=gamma_meV,
    I_nA=I_nA,
)

with open("data_base_0.pickle", "wb") as f:
    pickle.dump(theo, f)